In [1]:
import mysql.connector as mariadb
from random import randint

In [2]:

watering_duration = randint(300, 1800)
# seconds. Randomly calculate time between 5 and 30 mins, to test duration options

time_since_rules_check = 24 # if latest rules greater than this value, execute exception process
exception_hours_between_watering = 70

# Set db variables

db_host = 'hda.amahi.net'
db_host_port = '3306'
db_user = 'rpi'
db_pass = 'warm_me'
db = 'soil'

# Initialise variables

hold_watering = True
hours_since_last_rules = 999
watered = False
duration = 0.0

In [3]:
query_rules = """
SELECT
hold_watering,
TIMESTAMPDIFF(hour,ts,NOW())
FROM watering
order by ts
"""

con = mariadb.connect(host=db_host, port=db_host_port,
                      user=db_user, password=db_pass, database=db)
cur = con.cursor()

cur.execute(query_rules)

for row in cur:
    hold_watering = bool(row[0])
    hours_since_last_rules = row[1]

In [4]:
hold_watering, hours_since_last_rules

(True, 423)

In [5]:

query_exception = """
SELECT
TIMESTAMPDIFF(hour, ts, NOW()),
watered
FROM watering_records
WHERE watered in('1', 'True')
ORDER BY ts ASC
"""

con = mariadb.connect(host=db_host, port=db_host_port,
user=db_user, password=db_pass, database=db)

cur = con.cursor()

cur.execute(query_exception)

hours_since_last_water = 999

for row in cur:
    hours_since_last_water = row[0]

if hours_since_last_rules > time_since_rules_check:
    if hours_since_last_water > exception_hours_between_watering:
        hold_watering = False
    else:
        hold_watering = True



In [6]:
hours_since_last_water


790

In [7]:
hold_watering

False

In [8]:

import signal
import sys
import time
#import smbus
import mysql.connector as mariadb
from ast import literal_eval
import datetime
from random import randint

# Set control parameters

watering_duration = randint(300, 1800)
# seconds. Randomly calculate time between 5 and 30 mins, to test duration options

time_since_rules_check = 24 # if latest rules greater than this value, execute exception process
exception_hours_between_watering = 70

# Set db variables

db_host = 'hda.amahi.net'
db_host_port = '3306'
db_user = 'rpi'
db_pass = 'warm_me'
db = 'soil'

# Initialise variables

hold_watering = True
hours_since_last_rules = 999
watered = False
duration = 0.0

In [9]:

query_rules = """
SELECT
hold_watering,
TIMESTAMPDIFF(hour,ts,NOW())
FROM watering_rules
order by ts
"""

con = mariadb.connect(host=db_host, port=db_host_port,
                      user=db_user, password=db_pass, database=db)
cur = con.cursor()

cur.execute(query_rules)

for row in cur:
    hold_watering = bool(int(row[0]))
    hours_since_last_rules = row[1]


In [10]:
hold_watering


True

In [11]:
hours_since_last_rules

25

In [19]:

import signal
import sys
import time
import mysql.connector as mariadb
from ftplib import FTP
from ast import literal_eval
import xml.etree.ElementTree as ET
import datetime
from random import randint
import subprocess
import numpy as np
import pandas as pd


# Set variables

db_host = '192.168.0.10'
db_host_port = '3306'
db_user = 'rpi'
db_pass = 'warm_me'
db = 'soil'
#wifi_interface = "wlan0"
wifi_interface = "wlp59s0"

# Set control parameters


index_max = 2  # 1 = rest of day, 2 = tomorrow, so on
min_precip_threshold = 5  # mm expected
min_precip_prob_threshold = 0.6  # % chance of rain
min_hours_between_watering = 47
min_moisture_threshold = 3.0  # taken from moisture readings. Average over 12 hours

# Location parameters

# Beaumaris
forecast_id = "IDV10450.xml"
location = "VIC_PT042"


# Get BOM forecast xml

BomFtpHost = "ftp2.bom.gov.au"
BomFtpPort = 21
BomFtpForecastPath = "/anon/gen/fwo/"
retrieve_string = 'RETR ' + forecast_id


# def read_device_address():
#     try:
#         proc = subprocess.Popen(["ip", "a"], stdout=subprocess.PIPE, universal_newlines=True)
#         out, err = proc.communicate()
#         IP = ""
#         for line in out.split("\n"):
#             if("192.168" in line):
#                 strings = line.split(" ")
#                 full_device_address = strings[5]
#                 device_address = full_device_address[:-3]
#                 return(device_address)
#     except:
#         return("ERROR!-ifconfig")

def read_device_address():
    return("")


def convert_date(string):
    f = '%Y-%m-%dT%H:%M:%SZ'
    return datetime.datetime.strptime(string, f)


def create_list(index_max):
    listofzeros = [0.0] * index_max
    return listofzeros


# Get forecast XML

def get_forecast():
    ftp = FTP(BomFtpHost)
    ftp.login(user='anonymous', passwd='guest')
    ftp.cwd(BomFtpForecastPath)
    #ftp.retrlines('LIST')
    ftp.retrbinary(retrieve_string, open(forecast_id, 'wb').write)
    ftp.quit()
    return


# Parse xml

def parse_xml(arg):
    tree = ET.parse(arg)
    return tree


# Check whether forecast message is valid

def check_forecast_current(tree):

    issue_time = convert_date(tree.findtext("amoc/issue-time-utc"))
    expiry_time = convert_date(tree.findtext("amoc/expiry-time"))

    forecast_current = (issue_time <= datetime.datetime.utcnow() <= expiry_time)
    return forecast_current

# Extract precipitation details

# minimum forecast precipitation

def get_min_precip(index_max, tree):
    min_precip = create_list(index_max)
    root = tree.getroot()

    for loc in root.iter("area"):
        if loc.attrib['aac'] == location:
            for child in loc:
                for ind in range(0, index_max):
                    if child.attrib['index'] == str(ind):
                        for gc in child:
                            if(gc.attrib['type'] == 'precipitation_range'):
                                try:
                                    min_precip[ind] = np.mean([float(gc.text.split(' ')[0]), float(gc.text.split(' ')[2])])
                                except:
                                    min_precip[ind] = float(gc.text.split(' ')[0])


    return min_precip

# probability of precipitation 

def get_prob_precip(index_max, tree):
    prob_precip = create_list(index_max)

    root = tree.getroot()

    for loc in root.iter("area"):
        if loc.attrib['aac'] == location:
            for child in loc:
                for ind in range(0, index_max):
                    if child.attrib['index'] == str(ind):
                        for gc in child:
                            if(gc.attrib['type'] == 'probability_of_precipitation'):
                                prob_precip[ind] = float(gc.text.replace('%', ''))/100

    return prob_precip

# Last water

query_watering = """
SELECT
TIMESTAMPDIFF(hour,ts,NOW()),
watered
FROM watering_log
WHERE watered = 1
ORDER BY ts ASC
"""

def get_db_data(query, host, port, user, passwd, db):
    con = mariadb.connect(host=host, port=port, user=user, password=passwd, database=db)
    cur = con.cursor()
    cur.execute(query)

    output = 999
    
    for row in cur:
        output = row[0]

    return output



def watered_recently(hours_since_last_water, min_hours_between_watering):
    if hours_since_last_water < min_hours_between_watering:
        return True
    else:
        return False



# Determine whether to water based on rain forecast


def rain(min_precip, prob_precip, min_precip_threshold, min_precip_prob_threshold, index_max, forecast_current):
    
    output_tally = 0
        
    if forecast_current == True:
        for ind in range(0, index_max):
            if prob_precip[ind] > min_precip_prob_threshold and min_precip[ind] > min_precip_threshold:
                output_tally += 1
        if output_tally > 0:
            return True
        else:
            return False
        
# check average moisture

query_moisture = """
SELECT
avg(reading)
FROM soil_moisture
WHERE ts > DATE_SUB(now(), INTERVAL 12 HOUR)
"""


def moisture(av_moisture_L12H, min_moisture_threshold):
    if av_moisture_L12H > min_moisture_threshold:
        return True
    else:
        return False


def collate_rules(moisture_override, rain_override, watered_recently_override):
    func_list = [moisture_override, rain_override,watered_recently_override]
    if True in func_list:
        return True
    else:
        return False


# Add record to database


def insert_results(query, db_host, db_host_port, db_user, db_pass, db):

    con = mariadb.connect(host=db_host, port=db_host_port,
                       user=db_user, password=db_pass, database=db)
    cur = con.cursor()
    try:
        cur.execute(insert_stmt)
        con.commit()
    except:
        con.rollback()
    con.close()
    return


In [20]:
hours_since_last_water = get_db_data(query_watering, db_host, db_host_port, db_user, db_pass, db)

In [21]:
hours_since_last_water

999

In [14]:
query = """
SELECT
*
FROM watering_log
ORDER BY ts ASC
"""

In [15]:
con = mariadb.connect(host=db_host, port=db_host_port, user=db_user, password=db_pass, database=db)
cur = con.cursor()
df = cur.execute(query)



In [18]:
df =pd.read_sql(query, con)

InternalError: Unread result found

In [7]:
forecast_current = check_forecast_current(tree)


In [8]:
forecast_current

True

In [10]:
min_precip = get_min_precip(index_max, tree)


IndexError: list index out of range

In [21]:
min_precip = create_list(index_max)
root = tree.getroot()

for loc in root.iter("area"):
    if loc.attrib['aac'] == location:
        for child in loc:
            for ind in range(0, index_max):
                if child.attrib['index'] == str(ind):
                    for gc in child:
                        if(gc.attrib['type'] == 'precipitation_range'):
                            try:
                                min_precip[ind] = np.mean([float(gc.text.split(' ')[0]), float(gc.text.split(' ')[2])])
                            except:
                                min_precip[ind] = (float(gc.text.split(' ')[0]))


In [22]:
min_precip

[0.0, 0.0]